In [7]:
from pathlib import Path

import yaml
import trimesh
import h5py

from grasping_benchmarks.se3dif import Se3DifGraspPlanner
from grasping_benchmarks.base import CameraData

In [8]:
def load_acronym_mesh(
    dataset_path: Path, object_class: str, grasp_uuid: str
) -> trimesh.Trimesh:
    assert object_class in [
        p.name for p in (dataset_path / "grasps").iterdir()
    ], f"Object class {object_class} not found in dataset path {dataset_path}/grasps"

    grasp_file_path = list(
        (dataset_path / "grasps" / object_class).glob(
            f"{object_class}_{grasp_uuid}*.h5"
        )
    )

    assert len(grasp_file_path) != 0, f"Grasp file not found: {grasp_file_path}"
    assert len(grasp_file_path) < 2, f"Multiple grasp files found: {grasp_file_path}"
    grasp_file_path = grasp_file_path[0]

    grasp_data = h5py.File(grasp_file_path, "r")
    mesh_scale = grasp_data["object"]["scale"][()]
    mesh_file_path = (
        dataset_path
        / "meshes"
        / grasp_data["object"]["file"][()].decode("utf-8")[len("meshes") + 1 :]
    )

    mesh = trimesh.load_mesh(mesh_file_path)
    if type(mesh) == trimesh.scene.scene.Scene:
        mesh = trimesh.util.concatenate(mesh.dump())
    mesh = mesh.apply_translation(-mesh.centroid)
    mesh = mesh.apply_scale(mesh_scale)

    return mesh

mesh = load_acronym_mesh(Path("/home/data/"), "ScrewDriver", "28d")

In [9]:
with open(Path.cwd().parent / "cfg" / "base_config.yaml", "r") as f:
    cfg = yaml.load(f)

planner = Se3DifGraspPlanner(cfg)

/tmp/ipykernel_39345/3467568060.py:2: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  cfg = yaml.load(f)


In [10]:
camera_data = CameraData(
    pointcloud=mesh.vertices
)

In [13]:
grasps = planner.plan_grasp(camera_data)

None
